In [1]:
# Load initial packages and functions from own package
import scipy as sp
from src.PLS.utils import meta_mle
from src.PLS.utils import meta_mle_sir
import numpy as np
from src.PLS.metaSIR import meta_timed_sir
from decimal import Decimal, ROUND_UP
from src.PLS.metaSIR import basic_square_map
from src.PLS.metaSIR import straight_line_distances
from src.PLS.metaSIR import basic_kernel
import time

In [2]:
#Define the parameters and get the sample epidemic
R0 = 5
mle_b = []
start_time = time.time()
for i in [1]:
    seed = i
    N = 4
    test_map = basic_square_map(N)
    test_distances = straight_line_distances(test_map, N, scaling=10)
    gamma = 1
    X0 = [[999, 1, 0]]
    div = sum(X0[0])
    for i in range(N - 1):
        X0.append([1000, 0, 0])
    beta = R0 / div
    rng = np.random.default_rng(seed)
    tstep = 0.01
    tmax = 100
    tmax = float(Decimal(tstep) * (
                Decimal(meta_timed_sir(X0, beta, gamma, N, test_distances, basic_kernel, tmax, tstep, rng)) / Decimal(
            tstep)).quantize(1, rounding=ROUND_UP))
    rng = np.random.default_rng(seed)
    start_time = time.time()
    times_real, timed_real,causes = meta_mle_sir(X0, beta, gamma, N, test_distances, basic_kernel, tmax, tstep, rng)

In [3]:
#get length of reality
print(len(timed_real[0:7954]) == len(timed_real))

True


In [4]:
#Set up MLE array for different info
mle_b = []

In [5]:
subset_points = [10,25,50,100,250,500,750,1000,2000,3000,4000,5000,6000,7000,7954]

In [6]:
for subset in subset_points:
    #Define the optimisation function for the scipy solver
    def opt_mle(paras):
        return - meta_mle(paras[0]/div,gamma,timed_real[0:subset],times_real[0:subset],tmax,basic_kernel,test_distances,causes[0:subset])
    found = False
    for j in range(20):
        w = sp.optimize.minimize(opt_mle,x0=(4.0+j/10))
        if found == False:
            found = True
            best_b = w.x[0]
            best = w.fun
        elif w.fun > best:
            best_b = w.x[0]
    mle_b.append(best_b)

In [10]:
np.savetxt("MLE_graph.csv",mle_b,delimiter = ",")